In [134]:
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
import re 
import pyautogui
import pandas as pd

In [135]:
# Создаем таблицу ингредиентов. Только на первом проходе. 
# Потом она хранится где-то и пополняется уникальными ингредиентами
ingrDF = pd.DataFrame(columns=['ingrID', 'ingrNAME'])

In [136]:
def add_ingr(ingr_name, ingr_df):
    """
    Функция добавляет ингредиент с уникальным именем в таблицу ингредиентов

        Args:
            ingr_name (string): наименование ингредиента
            ingr_df (dataframe): таблица ингредиентов
 
        Returns:
            ingr_df (dataframe): таблица ингредиентов с добавленным уникальным ингредиентом    
    """
    # dataframe текущего ингредиента
    curr_ingr = pd.DataFrame(columns=['ingrID', 'ingrNAME'], index=range(1))
    curr_ingr.ingrNAME = ingr_name # наименование текущего ингредиента
    curr_ingr.ingrID = ingr_df.ingrID.count() # id текущего ингредиента
    # присоединение текущего ингредиента к исходному dataframe 
    ingr_df = pd.concat([ingr_df, curr_ingr], ignore_index=True)
    # удаление ингредиента, если такое наименование уже было в исходном dataframe  
    ingr_df.drop_duplicates(subset=['ingrNAME'], inplace=True)
    return ingr_df

In [142]:
url = pyautogui.prompt(text="Выберите рецепт на сайте https://www.russianfood.com и вставьте его адрес сюда", 
                                    title='Выбор рецепта' , 
                                    default='https://www.russianfood.com/recipes/recipe.php?rid=126032')

In [143]:
# Бефстроганов классический - в разделителях встречается тире
# https://www.russianfood.com/recipes/recipe.php?rid=116909

In [144]:
# Выполняем GET-запрос, содержимое ответа присваивается переменной response
response = requests.get(url, headers={
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер

In [145]:
page_recipe = page.find_all('span', class_="")
ingr_dict = {}
for i in range(len(page_recipe)):
    line = str(page_recipe[i].string)
    isCyrillic = bool(re.search('[а-яА-Я]', line))
    if isCyrillic:
        k, v = re.split(' [^а-яА-Я0-9] ', line, maxsplit=1)
#        k, v = re.split(' \W ', line, maxsplit=1) #этот вариант исключает и нижнее почеркивание
        ingr_dict[k] = v
        ingrDF = add_ingr(k, ingrDF)
ingr_dict    

{'Мясо говядины (филейная часть, кострец)': '500 г',
 'Лук репчатый крупный': '1 шт.',
 'Мука': '2 ч. ложки',
 'Сметана': '2-3 полных ст. ложки',
 'Масло растительное': 'для жарки',
 'Зелень укропа': 'по вкусу'}

In [146]:
ingrDF

,ingrID,ingrNAME
0,0,Вода (или бульон)
1,1,Свекла
2,2,Фасоль
3,3,Капуста белокочанная
4,4,Лук репчатый
5,5,Морковь
6,6,Картофель
7,7,Корень сельдерея
8,8,Корень петрушки
9,9,Чернослив
